# Homework 5
Jeffrey Xu   
11/02/2020  
Professor Mortazavi

## Problem 1: Gradient Boosting

## Problem 2: Random Forest

**Classifying benign vs malignant tumors:** We would like to classify if a tumor is benign or malign based on its attributes. We use data from the Breast Cancer Wisconsin Data Set of the UCI Machine Learning Repository: *https://archive.ics.uci.edu/m1/datasets/breast+cancer+wisconsin+(original)*. The input file is named "hw5_question2.csv" for our experiments. The rows of these files refer to the data samples, while the columns denote the deatures (columns 1-9) and the outcome variable (column 10), as described bellow:

1. Clump Thickness: discrete values {1,10}  
2. Uniformity of Cell Size: discrete values {1,10}  
3. Uniformity of Cell Shape: discrete values {1,10}  
4. Marginal Adhesion: discrete values {1,10}  
5. Single Epithelial Cell Size: discrete values {1,10}  
6. Bare Nuclei: discrete values {1,10}
7. Bland Chromatin: discrete values {1,10}  
8. Normal Nucleoli: discrete values {1,10}  
9. Mitoses: discrete values {1,10}  
10. Class: 2 for benign, 4 for malignant (this is the **outcome** variable)

**(a)** Compute the number of samples belonging to the benign and the number of samples belonging to the malignant case. What do you observe? Are the two classes equally represented in the data? Separate the data into a train (2/3 of the data) and a test (1/3 of the data) set. Make sure that both classes are represented with the same proportion in both sets. 

In [35]:
# Imports
import pandas as pd
from sklearn.model_selection import train_test_split

# Load data
data = pd.read_csv("hw5_question2.csv", header=None)

In [36]:
# Extract features and labels 
X = data.drop([9], axis=1)
y = data[9]

In [37]:
X

,0,1,2,3,4,5,6,7,8
0,5,1,1,1,2,1,3,1,1
1,5,4,4,5,7,10,3,2,1
2,3,1,1,1,2,2,3,1,1
3,6,8,8,1,3,4,3,7,1
4,4,1,1,3,2,1,3,1,1
...,...,...,...,...,...,...,...,...,...
678,3,1,1,1,3,2,1,1,1
679,2,1,1,1,2,1,1,1,1
680,5,10,10,3,7,3,8,10,2
681,4,8,6,4,3,4,10,6,1


In [38]:
y

0      2
1      2
2      2
3      2
4      2
      ..
678    2
679    2
680    4
681    4
682    4
Name: 9, Length: 683, dtype: int64

Now we need to look at the distribution of bengin and malignant cases. We first display the amount that is in each class. 

In [39]:
# Output the number of benign and malignant cases
print(y.value_counts())

2    444
4    239
Name: 9, dtype: int64


We notice that these two classes aren't necessarily distributed equally. We see that there are almost double the amount of benign cases compared to the number of malignant cases. However, now we want to split our dataset into a third testing and two thirds training while keeping this class distribution consistent through both datasets. 

In [42]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [43]:
print('Label distribution for training data:')
print(y_train.value_counts())
print()

print('Label distribution for testing data:')
print(y_test.value_counts())

Label distribution for training data:
2    302
4    155
Name: 9, dtype: int64

Label distribution for testing data:
2    142
4     84
Name: 9, dtype: int64


We notice by checking the split data, that the proportions of each class is about equal to the total proportions within the entire dataset. Therefore, we have properly analyzed and split the input data. 

**(b)** *Implement* two decision trees using the training samples. The splitting criterion for the first one should be the entropy, while for the second one should be the gini index. Plot the 10-fold stratified cross validated accuracy on the train and test data while the maximum depth in the tree increases for both splitting criteria. Do you observe any differences in practice? 

**(c)** *Feature Importance* Run a 10-fold stratified cross-validation for modeling this problem with a random forest model. You can choose to do a secondary 10-fold cross-validation on the training set in each fold to find the optimal number of trees and depth of each tree if you choose. Is this model more accurate (averaged over all 10 folds) than the decision trees? As ranked by the feature important, please provide a ranking of the features as their average position (standard deviation) of the rank order. Is this ranking it by Gini, Mean Decrease in Acccuracy, or something else? What should the final model's list of features be? Please justify your answer. 